Lista 2 - AMA - Luis Fernando Bastos Rego - 470043
#Apresentação dos modelos:

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
# implementações feitas por mim
import modelslib as model
import norm
import training

dataset = np.genfromtxt('./breastcancer.csv', delimiter=',')

fold_list = training.kfold(10, dataset)
logreg_param = []
nbg_param = []
gda_param = []

logreg_pred = []
nbg_pred = []
gda_pred = []

model_names = [ 'LR', 'GDA', 'NBG' ]
models_param = [ logreg_param, gda_param, nbg_param ]
model_preds = [ logreg_pred, gda_pred, nbg_pred ]

print(f"Número de amostras: {dataset.shape[0]}")
print(f"Número de dimensões: {dataset.shape[1]}")
print(f"Número de classes: {np.unique(dataset[:,[-1]]).shape[0]}")
print(f"Número de folds: {len(fold_list)}")

Número de amostras: 569
Número de dimensões: 31
Número de classes: 2
Número de folds: 10


* Para fazer a média e o desvio padrão de todas as 4 métricas para cada modelo, usaremos matrizes de dimensão 3 x 10:

In [9]:
# Inicializando métricas:
# Listas de Listas de métricas para cada modelo (são 3, divididos em 10-fold totalizando 30)
acurracy_matrix = [[],[],[]]
precision_matrix = [[],[],[]]
f1score_matrix = [[],[],[]]
recall_matrix = [[],[],[]]
# Lista para guardar os testes de cada fold, em formato de tupla (x,y):
fold_test_list = []

* Utilizando os folds com cada modelo:

In [10]:
for i in range(len(fold_list)):

    # Filtragem dos dados por cada fold: 
    train_dataset = dataset[fold_list[i].reshape(-1) == True] 
    test_dataset = dataset[fold_list[i].reshape(-1) == False]
    x_train = train_dataset[:,:-1]
    y_train = train_dataset[:,[-1]]
    x_test = test_dataset[:,:-1]
    y_test = test_dataset[:,[-1]]
    # Guarda-se o grupo de teste:
    fold_test_list.append((x_test,y_test))
    # Normalizando x:
    x_scaled = norm.normalize(x_train)
    x_test_scaled = norm.normalize(x_test)
    # Aplicando a coluna de 1s
    x_scaled = np.c_[np.ones((x_scaled.shape[0],1)), x_scaled]
    x_test_scaled = np.c_[np.ones((x_test_scaled.shape[0],1)), x_test_scaled]


    # Regressão Logística:
    w, iter_num, error_list = model.GD_logi(x_scaled, y_train)
    logreg_param.append(w)
    y_p = np.around(model.sigmoid(x_test_scaled @ w))
    logreg_pred.append(y_p)
    acurracy_matrix[0].append(metrics.accuracy_score(y_test, y_p))
    precision_matrix[0].append(metrics.precision_score(y_test, y_p, average='macro'))
    recall_matrix[0].append(metrics.recall_score(y_test, y_p, average='macro'))
    f1score_matrix[0].append(metrics.f1_score(y_test, y_p, average='macro'))

    
    # Análise de Discriminante Gaussiano:
    gda_p = model.gaussian_discriminant_analysis(x_train,y_train)
    gda_param.append(gda_p)
    y_p = model.predict_gda(x_test, gda_p[0], gda_p[1], gda_p[2])
    gda_pred.append(y_p)
    acurracy_matrix[1].append(metrics.accuracy_score(y_test, y_p))
    precision_matrix[1].append(metrics.precision_score(y_test, y_p, average='macro'))
    recall_matrix[1].append(metrics.recall_score(y_test, y_p, average='macro'))
    f1score_matrix[1].append(metrics.f1_score(y_test, y_p, average='macro'))


    # Naive Bayes Gaussiano:
    nbg_p = model.naive_bayes_gaussian(x_train,y_train)
    nbg_param.append(nbg_p)
    y_p = model.predict_nbg(x_test, nbg_p[0], nbg_p[1], nbg_p[2])
    nbg_pred.append(y_p)
    acurracy_matrix[2].append(metrics.accuracy_score(y_test, y_p))
    precision_matrix[2].append(metrics.precision_score(y_test, y_p, average='macro'))
    recall_matrix[2].append(metrics.recall_score(y_test, y_p, average='macro'))
    f1score_matrix[2].append(metrics.f1_score(y_test, y_p, average='macro'))

# Métricas de avaliação:

In [7]:
# Medições:
for i in range(len(model_names)):
    
    print(f'\nAverage score in 10-fold training for {model_names[i]}:\n')
    print(f'Accuracy:   {np.mean(acurracy_matrix[i]):.2f}, with standard deviation: {np.std(acurracy_matrix[i]):.2f};')
    print(f'Precision:  {np.mean(precision_matrix[i]):.2f}, with standard deviation: {np.std(precision_matrix[i]):.2f};')
    print(f'Recall:     {np.mean(recall_matrix[i]):.2f}, with standard deviation: {np.std(recall_matrix[i]):.2f};')
    print(f'F1:         {np.mean(f1score_matrix[i]):.2f}, with standard deviation: {np.std(f1score_matrix[i]):.2f};\n')



Average score in 10-fold training for LR:

Accuracy:   0.91, with standard deviation: 0.08;
Precision:  0.89, with standard deviation: 0.08;
Recall:     0.93, with standard deviation: 0.05;
F1:         0.90, with standard deviation: 0.09;


Average score in 10-fold training for GDA:

Accuracy:   0.96, with standard deviation: 0.01;
Precision:  0.94, with standard deviation: 0.02;
Recall:     0.96, with standard deviation: 0.02;
F1:         0.95, with standard deviation: 0.02;


Average score in 10-fold training for NBG:

Accuracy:   0.93, with standard deviation: 0.04;
Precision:  0.91, with standard deviation: 0.05;
Recall:     0.92, with standard deviation: 0.04;
F1:         0.91, with standard deviation: 0.04;



Caso seja interessante testar cada fold individualmente, pode-se usar esse fragmento de código apenas mudando as variáveis fold e model.

In [12]:
model = 0
fold = 0
print(f"Summary for the classifier {model_names[model]} (Fold {fold+1}) with accuracy {metrics.accuracy_score(fold_test_list[fold][1], model_preds[model][fold]):.2f}")
print(metrics.classification_report(fold_test_list[fold][1], model_preds[model][fold]))

Summary for the classifier LR (Fold 1) with accuracy 0.70
              precision    recall  f1-score   support

         0.0       1.00      0.63      0.77        46
         1.0       0.39      1.00      0.56        11

    accuracy                           0.70        57
   macro avg       0.70      0.82      0.67        57
weighted avg       0.88      0.70      0.73        57

